#### Installing Dependencies

In [1]:
#importing dependencies
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


#### Imports

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time
from PIL import Image

#### Mediapipe hand model initialization

In [3]:
#Draws line segments
mp_drawing = mp.solutions.drawing_utils
#mp hands model
mp_hands = mp.solutions.hands

#### Associating each landmark with its index

In [4]:
WRIST = 0
THUMB_CMC = 1
THUMB_MCP = 2
THUMB_IP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_DIP = 7
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_DIP = 11
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_DIP = 15
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_DIP = 19
PINKY_TIP = 20

![alt text](https://github.com/NamanBiyani06/HandPoseDetection/blob/main/images/hand_landmarks.png?raw=true)

#### Functions to dectect when a hand gesture is performed

##### Functions return gesture name depending if joint positions match the generalized positions of the specific gesture

In [5]:
# Is the model performing a fist
# Coordinate System starts at top left in cv2(Operations are reversed)
def fist_gesture(points):
    if points[INDEX_FINGER_TIP].y > points[INDEX_FINGER_MCP].y and \
       points[MIDDLE_FINGER_TIP].y > points[MIDDLE_FINGER_MCP].y and \
       points[RING_FINGER_TIP].y > points[RING_FINGER_MCP].y and \
       points[PINKY_TIP].y > points[PINKY_MCP].y:
        return True

def open_hand_gesture(points):
    if points[THUMB_TIP].y < points[THUMB_IP].y and \
       points[INDEX_FINGER_TIP].y < points[INDEX_FINGER_PIP].y and \
       points[MIDDLE_FINGER_TIP].y < points[MIDDLE_FINGER_PIP].y and \
       points[RING_FINGER_TIP].y < points[RING_FINGER_PIP].y and \
       points[PINKY_TIP].y < points[PINKY_PIP].y:
        return True

#### Opening a Computer Vision window and creating a loop to update the model each frame

In [15]:
#getting webcam feed #0
cap = cv2.VideoCapture(0)

cv2.startWindowThread()

#initializing gesture to prevent errors
gesture = "init"

#declaring following as "hands"
#detection threshold for intitial detection to be successful
#tracking threshold for tracking after initial detection
with mp_hands.Hands(min_detection_confidence = 0.8, min_tracking_confidence = 0.5) as hands:
    #reading through each frame
    while cap.isOpened():
        ret,frame = cap.read()

        #detections
        #recolours from BGR to RGB
        #passes in frame, outputs image and RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #image is flipped on the horizontal
        image = cv2.flip(image, 1)

        image.flags.writeable = False
        #detections
        results = hands.process(image)
        image.flags.writeable = True

        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Code to draw results on a PNG
        #image = Image.open('/Users/namanbiyani/Desktop/HandPoseDetection/images/grid.png')
        #image = np.asarray(image)

        #checking if we have results
        if results.multi_hand_landmarks:
            #putting landmarks into landmarks
            for landmark in results.multi_hand_landmarks:
                landmarks = landmark.landmark
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 204),
                thickness=2,
                circle_radius = 2),
                mp_drawing.DrawingSpec(color=(255, 102, 0),
                thickness=2,
                circle_radius=7))
            print(results.multi_handedness)

            
            #Obtaining current gesture
            if fist_gesture(landmarks):
                gesture = "Fist"
            elif open_hand_gesture(landmarks):
                gesture = "Palm Open"
            else:
                gesture = "NULL"

        
        # font used while displaying text
        font = cv2.FONT_HERSHEY_SIMPLEX
        # putting the current gesture on the window
        cv2.putText(image, gesture, (40, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Hand Tracking', image)
        cv2.setWindowProperty('Hand Tracking', cv2.WND_PROP_TOPMOST, 1)

        

        #close window if q is pressed
        if cv2.waitKey(10) & 0xff == ord('q'):
            cv2.waitKey(1)
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break

cv2.destroyAllWindows()
cap.release()



[classification {
  index: 0
  score: 0.9809434413909912
  label: "Left"
}
]
[classification {
  index: 0
  score: 0.9960464835166931
  label: "Left"
}
]
[classification {
  index: 1
  score: 0.9851076006889343
  label: "Right"
}
, classification {
  index: 0
  score: 0.9956160187721252
  label: "Left"
}
]
[classification {
  index: 0
  score: 0.9892488121986389
  label: "Left"
}
]
